In [17]:
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import glob
import os
import re

import spacy
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import en_core_web_sm
import re
import datetime
from afinn import Afinn
tqdm.pandas()

In [2]:
df_movie = pd.read_csv('/Users/yli17/Downloads/movie_data_2015_2020.csv', error_bad_lines=False)
df_movie.head()

,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,...,ordering_y,nconst,category,job,characters,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,0,tt1179933,movie,10 Cloverfield Lane,10 Cloverfield Lane,0.0,2016.0,NaN,103.0,"Action,Drama,Horror",...,10.0,nm6618222,producer,producer,NaN,Lindsey Weber,NaN,NaN,producer,"tt4530422,tt2660888,tt2548396,tt1179933"
1,1,tt4530422,movie,Overlord,Overlord,0.0,2018.0,NaN,110.0,"Action,Horror,Sci-Fi",...,9.0,nm6618222,producer,producer,NaN,Lindsey Weber,NaN,NaN,producer,"tt4530422,tt2660888,tt2548396,tt1179933"
2,2,tt1179933,movie,10 Cloverfield Lane,10 Cloverfield Lane,0.0,2016.0,NaN,103.0,"Action,Drama,Horror",...,1.0,nm0000422,actor,NaN,"[""Howard""]",John Goodman,1952.0,NaN,"actor,soundtrack,producer","tt0101410,tt1179933,tt1024648,tt1907668"
3,3,tt2406566,movie,Atomic Blonde,Atomic Blonde,0.0,2017.0,NaN,115.0,"Action,Thriller",...,3.0,nm0000422,actor,NaN,"[""Emmett Kurzfeld""]",John Goodman,1952.0,NaN,"actor,soundtrack,producer","tt0101410,tt1179933,tt1024648,tt1907668"
4,4,tt5968394,movie,Captive State,Captive State,0.0,2019.0,NaN,109.0,"Action,Horror,Sci-Fi",...,1.0,nm0000422,actor,NaN,"[""William Mulligan""]",John Goodman,1952.0,NaN,"actor,soundtrack,producer","tt0101410,tt1179933,tt1024648,tt1907668"


In [3]:
df_movie['startYear'] = df_movie['startYear'].astype(int)

df_movie = df_movie[df_movie["startYear"]==2015]

In [8]:
moviename = df_movie[df_movie["primaryTitle"].str.contains("The Big Short",na=False)]
moviename

,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,...,ordering_y,nconst,category,job,characters,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
401,401,tt1596363,movie,The Big Short,The Big Short,0.0,2015,NaN,130.0,"Biography,Comedy,Drama",...,10.0,nm0586969,producer,producer,NaN,Arnon Milchan,1944.0,NaN,"producer,miscellaneous,actor","tt1663202,tt2024544,tt0119488,tt2562232"
797,797,tt1596363,movie,The Big Short,The Big Short,0.0,2015,NaN,130.0,"Biography,Comedy,Drama",...,4.0,nm0000093,actor,NaN,"[""Ben Rickert""]",Brad Pitt,1963.0,NaN,"actor,producer,soundtrack","tt1210166,tt0114746,tt2935510,tt0356910"
816,816,tt1596363,movie,The Big Short,The Big Short,0.0,2015,NaN,130.0,"Biography,Comedy,Drama",...,8.0,nm0306890,producer,producer,NaN,Dede Gardner,NaN,NaN,"producer,location_management","tt4975722,tt1020072,tt2024544,tt7125860"
832,832,tt1596363,movie,The Big Short,The Big Short,0.0,2015,NaN,130.0,"Biography,Comedy,Drama",...,9.0,nm1250070,producer,producer,NaN,Jeremy Kleiner,NaN,NaN,"producer,miscellaneous","tt1020072,tt7125860,tt4975722,tt2024544"
1499,1499,tt1596363,movie,The Big Short,The Big Short,0.0,2015,NaN,130.0,"Biography,Comedy,Drama",...,5.0,nm0570912,director,NaN,NaN,Adam McKay,1968.0,NaN,"producer,writer,director","tt1596363,tt6266538,tt1386588,tt0478970"
1974,1974,tt1596363,movie,The Big Short,The Big Short,0.0,2015,NaN,130.0,"Biography,Comedy,Drama",...,2.0,nm0136797,actor,NaN,"[""Mark Baum""]",Steve Carell,1962.0,NaN,"actor,producer,writer","tt0425061,tt0386676,tt1100089,tt0405422"
2406,2406,tt1596363,movie,The Big Short,The Big Short,0.0,2015,NaN,130.0,"Biography,Comedy,Drama",...,3.0,nm0331516,actor,NaN,"[""Jared Vennett""]",Ryan Gosling,1980.0,NaN,"actor,soundtrack,producer","tt1856101,tt3783958,tt1120985,tt0332280"
2543,2543,tt1596363,movie,The Big Short,The Big Short,0.0,2015,NaN,130.0,"Biography,Comedy,Drama",...,6.0,nm1017488,writer,screenplay by,NaN,Charles Randolph,NaN,NaN,"writer,producer,executive","tt6394270,tt0289992,tt1596363,tt0758752"
4107,4107,tt1596363,movie,The Big Short,The Big Short,0.0,2015,NaN,130.0,"Biography,Comedy,Drama",...,1.0,nm0000288,actor,NaN,"[""Michael Burry""]",Christian Bale,1974.0,NaN,"actor,editorial_department,producer","tt0468569,tt1800241,tt0372784,tt0144084"
8533,8533,tt1596363,movie,The Big Short,The Big Short,0.0,2015,NaN,130.0,"Biography,Comedy,Drama",...,7.0,nm1034122,writer,based upon the book by,NaN,Michael Lewis,1960.0,NaN,writer,"tt1210166,tt3657682,tt0878804,tt1596363"


In [6]:
quotes = pd.read_csv('/Users/yli17/Downloads/movie_2015_crew_quotes.csv', error_bad_lines=False)
quotes.head()

,Unnamed: 0,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,23,2015-03-25-025189,how were you able to get that cat to do what y...,Cate Blanchett,['Q80966'],2015-03-25 21:26:54,1,"[['Cate Blanchett', '0.6709'], ['None', '0.329...",['http://www.pedestrian.tv/news/arts-and-cultu...,E
1,55,2015-11-10-031159,However the down side is we are very short of ...,Mark Williams,"['Q13427287', 'Q16192990', 'Q16194404', 'Q1861...",2015-11-10 12:21:00,1,"[['Mark Williams', '0.8594'], ['None', '0.1406']]",['http://www.elystandard.co.uk/help_needed_wit...,E
2,98,2015-09-08-000937,365 degrees is above the glass transition phas...,Tim Moore,"['Q16123784', 'Q16732824', 'Q7804002', 'Q78040...",2015-09-08 12:47:51,1,"[['Tim Moore', '0.8868'], ['None', '0.1132']]",['http://www.thefashionspot.com/beauty/637227-...,E
3,131,2015-01-16-022448,I am also one of the males in this male-domina...,Taapsee Pannu,['Q7672815'],2015-01-16 09:24:12,1,"[['Taapsee Pannu', '0.8301'], ['None', '0.0826...",['http://www.glamsham.com/movies/scoops/15/jan...,E
4,151,2015-04-12-012121,I am glad he's [ Sanjay Leela Bhansali ] chose...,Deepika Padukone,['Q159178'],2015-04-12 06:48:34,1,"[['Deepika Padukone', '0.6817'], ['Sonakshi Si...",['http://us.india.com/showbiz/omg-priyanka-cho...,E


In [12]:
quotes = quotes[quotes["quotation"].str.contains("The Big Short|the big short",na=False)]
quotes["primaryTitle"] = "The Big Short"
quotes.head()

,Unnamed: 0,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase,primaryTitle
15336,638594,2015-12-21-033867,"I thought, Ira Glass is no dope-I'll read that...",Adam McKay,"['Q350405', 'Q41538972']",2015-12-21 08:00:00,1,"[['Adam McKay', '0.7979'], ['None', '0.1463'],...",['http://www.kansascity.com/entertainment/movi...,E,The Big Short
18502,767877,2015-12-09-132127,"Well, definitely, Paul [ Rudd ], and we've got...",Peyton Reed,['Q944763'],2015-12-09 03:21:58,2,"[['Peyton Reed', '0.6529'], ['None', '0.1452']...",['http://movieweb.com/ant-man-2-wasp-genre-wri...,E,The Big Short
20905,864335,2015-12-17-090650,That film was supposed to be a sort-of comedic...,Adam McKay,"['Q350405', 'Q41538972']",2015-12-17 16:41:24,2,"[['Adam McKay', '0.6969'], ['None', '0.1961'],...",['http://www.edmontonsun.com/2015/12/17/direct...,E,The Big Short
25312,1048072,2015-12-02-055009,"In all fairness, he (Clooney) outbid me on Arg...",Brad Pitt,"['Q35332', 'Q373912']",2015-12-02 11:43:04,1,"[['Brad Pitt', '0.8497'], ['None', '0.142'], [...",['http://tribune.com.pk/story/1002292/brad-pit...,E,The Big Short
39046,1615344,2015-12-31-025532,I read The Big Short in one night and thought ...,Adam McKay,"['Q350405', 'Q41538972']",2015-12-31 03:39:00,1,"[['Adam McKay', '0.6069'], ['None', '0.2658'],...",['http://au.news.yahoo.com/thewest/entertainme...,E,The Big Short


In [34]:
df_total = pd.merge(quotes, moviename, on=["primaryTitle"])
#df_total.dtypes

In [35]:
afn = Afinn()

nlp = en_core_web_sm.load()

punctuations = string.punctuation
#stopwords = list(STOP_WORDS)

parser = nlp
def spacy_tokenizer(sentence):
    tokens = parser(sentence)
    tokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens ]
    tokens = [ word for word in tokens if word not in punctuations ]
    tokens = " ".join([i for i in tokens])
    return tokens

def count_words(s):
    return len(s.split())

def sentiment(s):
    return afn.score(s)

In [37]:
df_total["text_clean"] = df_total["quotation"].progress_apply(spacy_tokenizer)

df_total["text"] = df_total["text_clean"].astype('str')

  0%|          | 0/330 [00:00<?, ?it/s]

In [38]:
df_total["sentiment"] = df_total["text"].progress_apply(sentiment)
df_total["word_counts"] = df_total["text"].progress_apply(count_words)

  0%|          | 0/330 [00:00<?, ?it/s]

  0%|          | 0/330 [00:00<?, ?it/s]

In [40]:
df_total["word_counts"] = df_total[df_total["word_counts"]>5]

In [46]:
df_new = df_total[["text", "sentiment", "word_counts", "speaker", "date", "startYear", "primaryTitle", "genres", "Total Gross"]]
df_new = df_new.rename(columns={'date': 'quote_date', 'startYear': 'release_year', 'primaryTitle': 'movie_name'})
                                
df_ready = df_new.drop_duplicates()
df_ready.head()

,text,sentiment,word_counts,speaker,quote_date,release_year,movie_name,genres,Total Gross
0,i think ira glass be no dope i'll read that an...,6.0,638594,Adam McKay,2015-12-21 08:00:00,2015,The Big Short,"Biography,Comedy,Drama","$70,259,870"
10,well definitely paul rudd and we 've get andre...,3.0,767877,Peyton Reed,2015-12-09 03:21:58,2015,The Big Short,"Biography,Comedy,Drama","$70,259,870"
20,that film be suppose to be a sort of comedic p...,-1.0,864335,Adam McKay,2015-12-17 16:41:24,2015,The Big Short,"Biography,Comedy,Drama","$70,259,870"
30,in all fairness he clooney outbid i on argo bu...,7.0,1048072,Brad Pitt,2015-12-02 11:43:04,2015,The Big Short,"Biography,Comedy,Drama","$70,259,870"
40,i read the big short in one night and think ho...,-2.0,1615344,Adam McKay,2015-12-31 03:39:00,2015,The Big Short,"Biography,Comedy,Drama","$70,259,870"
